**Дубликаты**\
Дубликатами называют записи , для которых значения(всех или большинства)признаков совпадают

*Чем опасны дубликаты*\
Дублирующаяся информация никогда не приводит ни к чему хорошему.  в Data science  одинаковые записи не несут полезной информации и искажают реальную статистику. Модель несколько раз видит одно и тоже наблюдение и начинает подстраиваться под него . Если дубликатов много , это может стать большой проблемой при обучении 

**Обнаружение   и ликвидация дубликатов**\
Все зависит от того , что вы считаете дубликатами. Это могут быть записи где совпадают все признаки или только один. к примеру одинаковые записи могут имеnm одинаковый id\
Проверим, есть ли у нас такие записи: Для этого сравним число никальных хначений в столбце id с числом строк 

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
#импортируем базу данных сбера

sber_data =  pd.read_csv('D:\idi\DATA\sber_data.csv')
sber_data.head()


,id,full_sq,life_sq,floor,sub_area,preschool_quota,preschool_education_centers_raion,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,...,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,theater_km,museum_km,ecology,mosque_count_1000,price_doc
0,1,43,27.0,4.0,Bibirevo,5001.0,5,11065.0,5,0,...,0.637189,0.947962,0.177975,0.625783,0.628187,14.053047,7.389498,good,0,5850000
1,2,34,19.0,3.0,Nagatinskij Zaton,3119.0,5,6237.0,8,0,...,0.688796,1.072315,0.273345,0.967821,0.471447,6.829889,0.709260,excellent,0,6000000
2,3,43,29.0,2.0,Tekstil'shhiki,1463.0,4,5580.0,7,0,...,1.543049,0.391957,0.158072,3.178751,0.755946,4.273200,3.156423,poor,0,5700000
3,4,89,50.0,9.0,Mitino,6839.0,9,17063.0,10,0,...,0.934273,0.892674,0.236455,1.031777,1.561505,16.990677,16.041521,good,0,13100000
4,5,77,77.0,4.0,Basmannoe,3240.0,7,7770.0,9,0,...,0.077901,0.810801,0.376838,0.378756,0.121681,1.112486,1.800125,excellent,0,16331452


In [7]:
sber_data['id'].nunique()== sber_data.shape[0]


True

Вроде бы все в порядке : каждой записи соответвует свой уникальный идентификатор.  Но это еще не означает что в таблице нет дубликатов 

Тк как строка id  присваивает свой уникальный номер , поэому сама по себе каждая строка является уникальной. НО это не означает что др строкт не могут содержать в себе дубликаты 

Чтобы отследить дубликаты испльзуем метод dublicated()\
который возвращает булеву маску для фильтрации.Для записей у которых совпадают признаки переданные методу , он возвращает TRue , для остальных Falce\
У метода есть параметр Subset- список признаков по которым прозводится поиск дубликатов.  по умолчанию используются все столбцы в dataframe и ищуются полные дубликаты.


Найдем число полныз дубликатов в таблице sber_data. Предварительно создадим список столбцов  dupl_columns, по которым будем искать совпадения (все столбцы не включая id)\
создадим маску дубликатов с помощью метода dublicated() и произведем фильтрацию

In [9]:
dupl_columns=list(sber_data.columns)
dupl_columns.remove('id')
mask= sber_data.duplicated(subset=dupl_columns)
sber_duplicates= sber_data[mask]
print(f'Число найденнных дубликатов: {sber_duplicates.shape[0]}')

Число найденнных дубликатов: 562


Итак у нас 562 дубля .. которые являются полными дублями других записей. Ручной поиск совпадающих строк по 30т записей был практически бы не выполним , а с помощью пандас мы сделали это быстро\

Теперь нам необходимо от них  избавится. Для этого используем drop_dublicates,который уаляет повторяющиеся значения.

In [10]:
#создаем новую таблицу ,  которая будет являться копией старой.  с очищенными дубликатами
sber_dedupped = sber_data.drop_duplicates(subset=dupl_columns)
print(F'Результатирующее число записей: {sber_dedupped.shape[0]}')

Результатирующее число записей: 29909


**НЕИНФОРМАТИВНЫЕ ПРИЗНАКИ**\
Неинформативными называют признаки в которых большая часть строк содержит одинаковые значения, например пол клиентов в муж барбер шопе,  или наоборот признак в котором для большинства записей значения уникальны ( например, номер телефона клиента )

Такие признаки не играют роли при моделировании и лишь засоряют таблицу, увеличивая размерность данных . Чем увеличивают всемя обучения модели 

**обнаружение и ликвидация неинформативных признаков**

Чтобы считать признак неинформативным, необходимо задать порог (чаще всего это 95-99 процентов одинаковых или уникальных значений  )


**АЛгоритм**\
1. Создаем пустой список low_information_cols, куда будем добавлять признаки которые будем считать неинформативными 
2. в цикле пройдемся по всем именам столбцов в таблице и для каждого будем совершать след. действия\
- рассчитываем top_freg - наибольшую относительную частоту  с помощью метода value_counts() c параметром normalize = True. Метод вернет долю от общих  данных, которую занимает каждое уникальное значение в признаке\
рассчитываем nineque_ratio - отношение числа уникальных значений в столбце к размеру всего слотбца.Число уникальных значений в столбце получим с помощью метода nineque() а размер с помощью count()\
- срасвним каждое из полученных  чисел с пороговым значением ( у нас это 0,95) и добавим в список с неинформативными признаками, если условие истинно





In [11]:
low_information_cols=[]

#цикл по всем столбцам
for col in sber_data.columns:
    #наибольшая относительная частота в признаке
    top_freq= sber_data[col].value_counts(normalize=True).max()
    #доля уникальных  значений от размера признака
    nunique_ratio=sber_data[col].nunique()/ sber_data[col].count()
    #сравним наибольшую частоту с порогом 
    if top_freq> 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100,2)}% одинаковых  значений')
    #сравнимаем долю уникальных значений с порогом 
    if nunique_ratio> 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100,2)}% уникальных значений ')

id: 100.0% уникальных значений 
oil_chemistry_raion: 99.03% одинаковых  значений
railroad_terminal_raion: 96.27% одинаковых  значений
nuclear_reactor_raion: 97.17% одинаковых  значений
big_road1_1line: 97.44% одинаковых  значений
mosque_count_1000: 98.08% одинаковых  значений


Мы нашли шесть неинформативных признака. теперь их  дропнем. передав результатирующий список в его аргументы 


In [12]:
information_sber_data=sber_data.drop(low_information_cols,axis=1)
print(f'результатирующее число признаков: {information_sber_data.shape[1]}')

результатирующее число признаков: 55


Однако всегда следует скептически относиться к результатам, которые предоставил алгоритм поиска неинформативных признаков. В противном случае можно лишиться важных данных. 

Например, в ваших данных есть признак цены акций, которая всё время меняется. В каждый момент времени её цена уникальна, то есть число уникальных значений в столбце совпадает с размером столбца. Казалось бы, по критерию информативности это кандидат на выбывание, но если мы рассматриваем задачу предсказания спроса на эти самые акции, то очевидно, что цена — это очень важный фактор, несмотря на свою уникальность. Именно цена на акции определяет спрос на них.

Могут возникнуть ситуации, когда присутствует огромная разница в частотах для каждого уникального значения в столбце, но это вовсе не значит, что признак неинформативен. Например, у нас могут быть данные о различных переводах между счетами пользователей. В этих данных может быть очень важный признак мошенничества (два возможных значения — да и нет). Как правило, соотношение между мошенниками и обычными пользователями около — 2/98 % (мошенников 1-2 %, а обычных пользователей — 98-99 %). Но стоит ли удалять данный признак из данных зависит от поставленной перед нами задачи.

Будьте внимательны и рассудительны при поиске неинформативных признаков. Лучшее решение — для начала использовать все признаки для построения базовой модели, а затем уже выбирать те, которые обладают наибольшей информативностью.

Более подробно мы рассмотрим данные нюансы в блоке по машинному обучению.

**Важность признаков**\
Информативности признаков определяется не только числом уникальныз значаений,  но и влиянием на целевой признак ( что мы хотим предсказать) это и есть важность признака\
Признаки с низкой важностью - называются нерелевантными 